Here is the code to release the queries with the parameters you have selected using the OpenDP library

In [ ]:
# notebook imports
import opendp.prelude as dp
import math
import pandas as pd
import numpy as np
dp.enable_features("contrib")
dp.enable_features("honest-but-curious")
from typing import List

## Query: sex_histogram

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/PUMS_california_demographics_1000.csv')

In [ ]:
query_info = {"export_selected_query": "sex_histogram", "export_mechanism": "gaussian", "export_epsilon": 0.25, "url": "https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/PUMS_california_demographics_1000.csv", "export_data_type": "categorical", "export_delta": 1e-06}

In [ ]:
def at_delta(meas, delta):
    meas = dp.c.make_zCDP_to_approxDP(meas)
    return dp.c.make_fix_delta(meas, delta)

In [ ]:
# Load data and get parameters
df = pd.read_csv('https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/PUMS_california_demographics_1000.csv')
query_type = 'sex_histogram'
mechanism = 'gaussian'
epsilon = 0.25
column_name = 'sex'
categories = [0, 1]
column_type = df[column_name].dtype

In [ ]:
delta = 1e-06
t_hist = (
    dp.t.make_split_dataframe(separator=",", col_names=list(df.columns)) >>
    dp.t.make_select_column(column_name, str) >>
    dp.t.then_cast_default(int) >>
    dp.t.then_count_by_categories(categories=categories, MO=dp.L2Distance[float])
)
m_hist = dp.binary_search_chain(
    lambda s: at_delta(t_hist >> dp.m.then_gaussian(scale=s), delta), 
    d_in=1, 
    d_out=(epsilon, delta))
sensitive_gaussian_counts = m_hist(df.to_csv(index=False, header=False))

In [ ]:
print("Noisy Gaussian Counts:", sensitive_gaussian_counts[:-1])